In [1]:
"Hello world"

'Hello world'

In [50]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

In [9]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

In [14]:
@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny {city}"

In [25]:
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id : str

In [17]:
@tool
def get_user_location(runtime : ToolRuntime[Context]) -> str:
    """get user location"""
    user_id = runtime.context.user_id
    return "Colombo" if user_id == "1" else "SF"

In [18]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv("./.env")

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [19]:
model.invoke("Hello world")

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--d33f4c85-458a-49ae-9532-f11b4d498cd7-0', usage_metadata={'input_tokens': 3, 'output_tokens': 43, 'total_tokens': 46, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 33}})

In [35]:
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    punny_response:str
    weather_condition : str | None = None

In [27]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [23]:
from langchain.agents import create_agent

In [36]:
agent = create_agent(
    model = model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location , get_weather],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

In [54]:
config = {"configurable" : {"thread_id" : "1"}}

response = agent.invoke(
    {"messages" : [{"role" : "user" , "content" : "what is the my location?"}]},
    config=config,
    context=Context(user_id="1")
)


In [53]:
print(response['structured_response'])

ResponseFormat(punny_response="You're in SF! Hope you're not feeling too *foggy* about where you are!", weather_condition=None)
